In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.interpolate import make_interp_spline
from scipy.stats import pearsonr
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

## LNTWS Wörterbuch anwenden

Laden der bereinigten Artikel und der expansiven sowie restriktiven Uni- und Bigramme.

In [ ]:
df = pd.read_csv('bereinigte_artikel_spezifisch.csv')
terms_df = pd.read_csv('lntws_wörterbuch.csv', delimiter=';')

expansionary_terms = terms_df['expansive_terme'].dropna().tolist()
contractionary_terms = terms_df['restriktive_terme'].dropna().tolist()
expansionary_terms = [term.replace("_", " ") for term in expansionary_terms]
contractionary_terms = [term.replace("_", " ") for term in contractionary_terms]
print(len(df))

In [ ]:
# Definieren der Ereignisse und deren Zeiträume in datetime
events = {
    "Dotcom-Blase": ("2001-02-01", "2003-06-30"),
    "Weltwirtschaftskrise": ("2008-01-01", "2009-04-30"),
    "Coronakrise": ("2019-09-01", "2020-04-30")
}

Zählen der Terme in den Artikeln.

In [ ]:
# Funktion zur Extraktion der Terme mit Häufigkeiten
def extract_terms_with_frequencies(text, term_list, ngram_range=(1, 2)):
    vectorizer = CountVectorizer(ngram_range=ngram_range, vocabulary=term_list, token_pattern=r'\b\w+\b')
    term_counts = vectorizer.fit_transform([text]).toarray().flatten()
    term_freq = {term: count for term, count in zip(term_list, term_counts) if count > 0}
    return term_freq

# Beispiel: Konvertiere die Spalte "Datum" in datetime
df['Datum'] = pd.to_datetime(df['Datum'], format='%Y-%m-%d')

# Liste zur Speicherung der Ergebnisse
results = []

for index, row in df.iterrows():
    article = row['text_preprocessed_specific']
    date = row['Datum']  # Datum des Artikels
    year = row['datum_jahr'] # Jahr des Artikels
    quarter = row['datum_quartal'] # Quartal des Artikels
    month = row['Datum'].month  # Monat des Artikels
    total_words = row['länge_bereinigt_spezifisch']
    article_id = index + 1  # Nummerierung der Artikel

    # Gesamtanzahl der Wörter in den jeweiligen Artikeln
    #total_words = len(article.split())

    # Extrahiere expansive und kontraktive Terme mit ihren Häufigkeiten
    expansive_terms_freq = extract_terms_with_frequencies(article, expansionary_terms, ngram_range=(1, 2))
    contractionary_terms_freq = extract_terms_with_frequencies(article, contractionary_terms, ngram_range=(1, 2))

    # Berechne die Summe der expansiven und kontraktiven Terme
    expansive_count = sum(expansive_terms_freq.values())
    contractionary_count = sum(contractionary_terms_freq.values())

    # Gesamtanzahl der gefundenen Terme
    total_terms_found = expansive_count + contractionary_count

    # Berechnung des Sentiment Scores
    sentiment_score = (expansive_count - contractionary_count) / total_words if total_words > 0 else 0

    # Füge die Ergebnisse in die Liste ein
    results.append({
        'Datum': date,
        'Jahr': year,
        'Quartal': quarter,
        'Monat': month,
        'Gesamtanzahl Wörter': total_words,
        'Insgesamt gefundene Terme': total_terms_found,
        'Sentiment Score': sentiment_score,
        'Gefundene expansive Terme': expansive_terms_freq,
        'Gefundene kontraktive Terme': contractionary_terms_freq
    })

# Konvertiere die Ergebnisse in einen DataFrame
results_df = pd.DataFrame(results)

# Beispielhafte Ausgabe zur Kontrolle
print(results_df.head())

Überprüfen, welche Terme am häufigsten im gesamten Korpus auftreten.

In [ ]:
# Summieren der Häufigkeiten der expansiven Terme über den gesamten Datensatz
expansive_term_totals = {}
for result in results:
    for term, freq in result['Gefundene expansive Terme'].items():
        expansive_term_totals[term] = expansive_term_totals.get(term, 0) + freq

# Summieren der Häufigkeiten der kontraktiven Terme über den gesamten Datensatz
contractionary_term_totals = {}
for result in results:
    for term, freq in result['Gefundene kontraktive Terme'].items():
        contractionary_term_totals[term] = contractionary_term_totals.get(term, 0) + freq

# Konvertieren der Ergebnisse in DataFrames
expansive_terms_df = pd.DataFrame(list(expansive_term_totals.items()), columns=['expansive_terme', 'expansiv_häufigkeit'])
contractionary_terms_df = pd.DataFrame(list(contractionary_term_totals.items()), columns=['restriktive_terme', 'restriktiv_häufigkeit'])

# Verbinden beider Tabellen zu einer Übersichtstabelle
final_df = pd.concat([expansive_terms_df, contractionary_terms_df], axis=1)

# Beispielhafte Ausgabe der Übersichtstabelle
print(final_df.head())

In [ ]:
# Exportieren der Häufigkeiten der einzelnen restriktiven und expansiven Terme
final_df.to_csv('termhäufigkeit_artikel.csv', index=False, sep=';')

### Filtern der Artikel
Es werden nur die Artikel behalten, die mindestens einen expansiven oder restriktiven Term enthalten.

In [ ]:
# Filtern nur der Artikel, die mindestens einen expansiven oder kontraktiven Begriff enthalten
filtered_results_df = results_df[
    (results_df['Gefundene expansive Terme'].apply(len) > 0) |
    (results_df['Gefundene kontraktive Terme'].apply(len) > 0)
]

# Filtern der ursprünglichen Datei basierend auf dem Index der relevanten Artikel
filtered_df = df.loc[filtered_results_df.index]

In [ ]:
# Speichern der gefilterten Artikel in einer neuen Datei
filtered_df.to_csv('artikel_relevant.csv', index=False, encoding='utf-8')

print(f"Gefilterte Artikel wurden gespeichert. Anzahl der Artikel: {filtered_df.shape[0]}")

In [ ]:
# Anzahl der herausgefallenen Artikel
dropped_articles = results_df[
    (results_df['Gefundene expansive Terme'].apply(len) == 0) &
    (results_df['Gefundene kontraktive Terme'].apply(len) == 0)
]

print(f"Anzahl der herausgefallenen Artikel: {dropped_articles.shape[0]}")

Monatliches und quartalsweises Aggregieren der gefilterten Sentimentwerte.

In [ ]:
# Aggregieren der Scores quartalsweise (nur gefilterte Artikel)
sentiment_quarterly = filtered_results_df.groupby(['Jahr', 'Quartal'])[['Sentiment Score']].mean().reset_index()

# Aggregieren der Scores monatlich (nur gefilterte Artikel)
sentiment_monthly = filtered_results_df.groupby(['Jahr', 'Monat'])[['Sentiment Score']].mean().reset_index()

In [ ]:
# Speichern der Ergebnisse in einer CSV-Datei
sentiment_quarterly.to_csv('artikel_sentiment_lntws_quartal.csv', index=False)
sentiment_monthly.to_csv('artikel_sentiment_lntws_monat.csv', index=False)

#### Artikel mit extremen Sentimentwerten (LNTWS)

In [ ]:
# Bestimmen der 3 Monate mit den höchsten Sentimentwerten
top_3_months = sentiment_monthly.nlargest(3, 'Sentiment Score')[['Datum', 'Sentiment Score']]

# Bestimmen der 3 Monate mit den niedrigsten Sentimentwerten
bottom_3_months = sentiment_monthly.nsmallest(3, 'Sentiment Score')[['Datum', 'Sentiment Score']]

# Ausgabe der Ergebnisse
print("Top 3 Monate mit höchsten Sentimentwerten:")
print(top_3_months)

print("\nTop 3 Monate mit niedrigsten Sentimentwerten:")
print(bottom_3_months)

### Diagramm erstellen
Erstellen eines Diagramms zur Veranschaulichung der Entwicklung des Sentiments in den Wirtschaftsartikeln.

In [ ]:
sentiment_quarterly = pd.read_csv('artikel_sentiment_lntws_quartal.csv')
sentiment_monthly = pd.read_csv('artikel_sentiment_lntws_monat.csv')

In [ ]:
# Konvertieren der Quartale und Monate in datetime-Format
sentiment_quarterly['Datum'] = pd.PeriodIndex(
    sentiment_quarterly['Jahr'].astype(str) + 'Q' + sentiment_quarterly['Quartal'].astype(str), freq='Q'
).to_timestamp()

sentiment_monthly['Datum'] = pd.to_datetime(
    sentiment_monthly['Jahr'].astype(str) + '-' + sentiment_monthly['Monat'].astype(str).str.zfill(2) + '-01')

# Filtern der Sentiment-Daten
end_date = pd.to_datetime("2021-09-30")
sentiment_quarterly = sentiment_quarterly[sentiment_quarterly['Datum'] <= end_date]
sentiment_monthly = sentiment_monthly[sentiment_monthly['Datum'] <= end_date]

# Erstellen des Plots
plt.figure(figsize=(12, 6))

# Plot für quartalsweises Sentiment
plt.plot(sentiment_quarterly['Datum'], sentiment_quarterly['Sentiment Score'], label='LNTWS Sentiment der Artikel (quartalsweise)', color='blue')

# Plot für monatliches Sentiment
plt.plot(sentiment_monthly['Datum'], sentiment_monthly['Sentiment Score'], label='LNTWS Sentiment der Artikel (monatlich)', color='cornflowerblue')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=sentiment_quarterly['Datum'].min(),
                           end=sentiment_quarterly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Score', fontsize=12)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.tight_layout()

# Speichern der Artikel
plt.savefig('artikel_sentiment_relevant.png', format='png', dpi=600)

plt.show()

### Vergleich mit BIP

In [ ]:
# Lade die Datei mit den GDP Change-Daten
gdp_change_data = pd.read_csv('bip_veränderung.csv')

# Funktion zur Umwandlung von Quartalen in Enddatum
def convert_quarter_to_date(year, quarter):
    if quarter == "1. Quartal":
        return f"{year}-03-31"
    elif quarter == "2. Quartal":
        return f"{year}-06-30"
    elif quarter == "3. Quartal":
        return f"{year}-09-30"
    elif quarter == "4. Quartal":
        return f"{year}-12-31"
    else:
        return None  # Für unerwartete Werte

# Erstellen einer neue Spalte 'Date' mit dem Enddatum des Quartals
gdp_change_data['Date'] = gdp_change_data.apply(
    lambda row: convert_quarter_to_date(row['Year'], row['Quarter']),
    axis=1
)

# Konvertieren der neuen 'Date'-Spalte in datetime-Format
gdp_change_data['Date'] = pd.to_datetime(gdp_change_data['Date'], format='%Y-%m-%d')

# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")
gdp_change_data = gdp_change_data[gdp_change_data['Date'] <= end_date]
sentiment_quarterly = sentiment_quarterly[sentiment_quarterly['Datum'] <= end_date]

# Sortieren der BIP-Daten nach Datum
gdp_change_data = gdp_change_data.sort_values(by='Date')

# Erstellen der Grafik
fig, ax1 = plt.subplots(figsize=(12, 6))

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=sentiment_quarterly['Datum'].min(),
                           end=sentiment_quarterly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Plot für Sentiment
line1, = ax1.plot(sentiment_quarterly['Datum'], sentiment_quarterly['Sentiment Score'],
                  label='LNTWS Sentiment der Artikel (quartalsweise)', color='blue')
ax1.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax1.set_ylabel('Sentiment-Score', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(alpha=0.3)

# Plot für die quartalsweise Veränderung des BIPs
ax2 = ax1.twinx()
line2, = ax2.plot(gdp_change_data['Date'], gdp_change_data['GDP Change %'],
                  label='Veränderung des BIP in % (quartalsweise)', color='darkorange', linestyle='--')
ax2.set_ylabel('BIP-Veränderung (%)', color='darkorange')
ax2.tick_params(axis='y', labelcolor='darkorange')

# Anpassen der y-Achse
ax2.set_ylim([gdp_change_data['GDP Change %'].min() - 1, gdp_change_data['GDP Change %'].max() + 1])  # Example: add 1 to min/max for padding

for event, (start_date, end_date) in events.items():
    ax1.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3, label=f'{event} (Krise)')

# Kombinieren der Legenden
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

# Titel und Layout
# plt.title('', fontsize=14)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_vergleich_bip_relevant.png', format='png', dpi=600)

# Zeige den Plot
plt.show()

#### Korrelation mit BIP

In [ ]:
# Laden der quartalsweisen Daten
sentiment_quarterly = pd.read_csv('artikel_lntws_relevant_quartal.csv')
gdp_change_data = pd.read_csv('bip_veränderung.csv')

In [ ]:
# Erstellen eines konsistenten Quartalsformats für die BIP-Daten
def convert_quarter_to_numeric(quarter_str):
    """Konvertiert 'X. Quartal' in eine numerische Darstellung (1, 2, 3, 4)."""
    return int(quarter_str.split('.')[0])

gdp_change_data['Quarter'] = gdp_change_data['Quarter'].apply(convert_quarter_to_numeric)

# Zusammenführen beider Datensätze anhand von Jahr und Quartal zusammen
merged_data = pd.merge(
    sentiment_quarterly[['Jahr', 'Quartal', 'Sentiment Score']],
    gdp_change_data[['Year', 'Quarter', 'GDP Change %']],
    left_on=['Jahr', 'Quartal'],
    right_on=['Year', 'Quarter']
)

# Berechnen der Korrelation zwischen Sentiment Score und GDP Change %
correlation = (merged_data['Sentiment Score']).corr(merged_data['GDP Change %'])

print(f"Korrelation zwischen Sentiment-Score und BIP-Veränderung: {correlation:.2f}")

# Streudiagramm der Korrelation
plt.figure(figsize=(8, 6))
plt.scatter(merged_data['Sentiment Score'], merged_data['GDP Change %'], alpha=0.7, color='blue')
plt.title('Zusammenhang zwischen Sentiment-Score und BIP-Veränderung', fontsize=14)
plt.xlabel('Sentiment-Score')
plt.ylabel('BIP-Veränderung (%)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

#### Lag-Analyse

In [ ]:
# Funktion der Lag-Analyse
def lag_analysis(data, max_lag, sentiment_column, gdp_column):
    lag_results = []
    for lag in range(-max_lag, max_lag + 1):  # Von negativem bis positivem Lag
        if lag < 0:  # Sentiment führt das BIP an
            lagged_sentiment = data[sentiment_column].shift(-lag)
            correlation = data[gdp_column].corr(lagged_sentiment)
        elif lag > 0:  # BIP führt das Sentiment an
            lagged_gdp = data[gdp_column].shift(lag)
            correlation = data[sentiment_column].corr(lagged_gdp)
        else:  # Kein Lag, normale Korrelation
            correlation = data[sentiment_column].corr(data[gdp_column])
        lag_results.append({'Lag': lag, 'Correlation': correlation})

    return pd.DataFrame(lag_results)

# Durchführen der Lag-Analyse
max_lag = 4  # Analysiere bis zu 4 Quartale vor/nach
lag_results = lag_analysis(merged_data, max_lag, 'Sentiment Score', 'GDP Change %')

# Zeigen der Ergebnisse
# print(lag_results)

# Plot der Lag-Analyse
plt.figure(figsize=(10, 6))
plt.plot(lag_results['Lag'], lag_results['Correlation'], marker='o', linestyle='-', color='blue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
plt.title('Lag-Analyse: Korrelation zwischen Sentiment und BIP-Veränderung', fontsize=14)
plt.xlabel('Lag (Quartale)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_bip_lag_relevant.png', format='png', dpi=600)

# Anzeigen des Plots
plt.show()

Analyse nur in Krisenzeiten (und zwei Quartale vor- und nachher)

In [ ]:
# Datumskonvertierungen
sentiment_quarterly['Datum'] = pd.PeriodIndex(
    sentiment_quarterly['Jahr'].astype(str) + 'Q' + sentiment_quarterly['Quartal'].astype(str), freq='Q'
).to_timestamp()

# Konsistentes Quartalsformat für BIP-Daten
def convert_quarter_to_numeric(quarter_str):
    return int(quarter_str.split('.')[0])

gdp_change_data['Quarter'] = gdp_change_data['Quarter'].apply(convert_quarter_to_numeric)

# Merge des Sentiments und der BIP-Daten
merged_data = pd.merge(
    sentiment_quarterly[['Datum', 'Jahr', 'Quartal', 'Sentiment Score']],
    gdp_change_data[['Year', 'Quarter', 'GDP Change %']],
    left_on=['Jahr', 'Quartal'],
    right_on=['Year', 'Quarter']
)

def extend_event_period(start_date, end_date, quarters=2):
    """Erweitert einen Zeitraum um eine definierte Anzahl an Quartalen davor und danach."""
    start = pd.Timestamp(start_date) - pd.offsets.QuarterBegin(quarters)
    end = pd.Timestamp(end_date) + pd.offsets.QuarterEnd(quarters)
    return start, end

# Lag-Analyse-Funktion
def lag_analysis(data, max_lag, sentiment_column, gdp_column):
    lag_results = []
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            lagged_sentiment = data[sentiment_column].shift(-lag)
            correlation = data[gdp_column].corr(lagged_sentiment)
        elif lag > 0:
            lagged_gdp = data[gdp_column].shift(lag)
            correlation = data[sentiment_column].corr(lagged_gdp)
        else:
            correlation = data[sentiment_column].corr(data[gdp_column])
        lag_results.append({'Lag': lag, 'Correlation': correlation})
    return pd.DataFrame(lag_results)

max_lag = 4

all_event_periods = []

for start, end in events.values():
    extended_start, extended_end = extend_event_period(start, end)
    all_event_periods.append((extended_start, extended_end))

# Kombinierte Daten für alle Krisenzeiträume filtern
combined_data = pd.DataFrame()

for start, end in all_event_periods:
    filtered_data = merged_data[
        (merged_data['Datum'] >= start) & (merged_data['Datum'] <= end)
    ]
    combined_data = pd.concat([combined_data, filtered_data])

# Lag-Analyse für kombinierte Daten durchführen
combined_lag_results = lag_analysis(combined_data, max_lag, 'Sentiment Score', 'GDP Change %')

# Plot der kombinierten Lag-Analyse
plt.figure(figsize=(10, 6))
plt.plot(combined_lag_results['Lag'], combined_lag_results['Correlation'], marker='o', linestyle='-', color='blue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
# plt.title('Lag-Analyse: Alle Krisenzeiträume kombiniert', fontsize=14)
plt.xlabel('Lag (Quartale)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_bip_lag_relevant_krisen.png', format='png', dpi=600)

# Anzeigen des Plots
plt.show()

#### Regressionsgrade

In [ ]:
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [ ]:
# Regressionsanalyse
X = merged_data[['Sentiment Score']]
y = merged_data['GDP Change %']

model = LinearRegression()
model.fit(X, y)

print("Koeffizienten:", model.coef_)
print("Intercept:", model.intercept_)
print("R^2-Score:", model.score(X, y))

In [ ]:
# Scatterplot für Latifi et al. Score
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Sentiment Score', y='GDP Change %', data=merged_data, label='Datenpunkte', color='blue')  # Farbe auf darkgreen setzen

# Berechnen der Regressionsgrade (für Sentiment Score)
x_values = np.linspace(merged_data['Sentiment Score'].min(), merged_data['Sentiment Score'].max(), 100)
y_pred = model.intercept_ + model.coef_[0] * x_values  # Nutze nur den Koeffizienten für 'Sentiment Score'
plt.plot(x_values, y_pred, color='red', label='Regressionsgrade')

# plt.title('LNTWS Score vs. BIP Veränderung %')
plt.xlabel('LNTWS Sentiment der Artikel (quartalsweise)')
plt.ylabel('BIP Veränderung %')
plt.legend()

# Speichern der Grafik
plt.savefig('regression_artikel_relevant.png', format='png', dpi=600)

plt.show()

## BPWS Wörterbuch anwenden

Als Grundlage diente das Wörterbuch von Bannier et al., (2019) (https://www.uni-giessen.de/de/fbz/fb02/forschung/research-networks/bsfa/textual_analysis/index)

In [ ]:
# Laden der Artikel
articles = pd.read_csv("artikel_relevant.csv")
terms_df = pd.read_csv('bpws_wörterbuch.csv', delimiter=';')

positives = terms_df['lemma_positiv'].dropna().tolist()
negatives = terms_df['lemma_negativ'].dropna().tolist()

In [ ]:
# Funktion zur Berechnung des Sentiment-Scores
def calculate_sentiment(text, positives, negatives):
    lemmas = text.split()

    # Zählen positiver und negativer Wörter
    positive_count = sum(1 for lemma in lemmas if lemma in positives)
    negative_count = sum(1 for lemma in lemmas if lemma in negatives)

    # Berechnen des Sentiment-Scores
    word_count = len(lemmas)
    if word_count == 0:
        return 0
    bpw_score = (positive_count - negative_count) / word_count
    return bpw_score

# Berechnen des bpw-Scores für jeden Artikel
articles["BPW Score"] = articles["text_preprocessed_specific"].apply(
    lambda text: calculate_sentiment(text, positives, negatives)
)

# Beispielhafte Ausgabe zur Kontrolle
print(articles[["Datum", "BPW Score"]].head())

In [ ]:
# Konvertieren des Datums in ein datetime-Format mit dem Format 'DD.MM.YYYY'
articles['Datum'] = pd.to_datetime(articles['Datum'], format='%Y-%m-%d')

# Quartalsweise aggregieren und normalisieren
articles['Jahr'] = articles['Datum'].dt.year
articles['Quartal'] = articles['Datum'].dt.quarter  # Get quarter as numeric
articles['Monat'] = articles['Datum'].dt.month

# Aggregieren des Sentiments quartalsweise (und Durchschnitt bilden)
bpw_quarterly = articles.groupby(['Jahr', 'Quartal'])[['BPW Score']].mean().reset_index()
bpw_monthly = articles.groupby(['Jahr', 'Monat'])[['BPW Score']].mean().reset_index()

bpw_quarterly.head()

In [ ]:
# Speichere die Ergebnisse in einer neuen CSV-Datei
bpw_quarterly.to_csv('artikel_bpws_relevant_quartal.csv', index=False)
bpw_monthly.to_csv('artikel_bpws_relevant_monat.csv', index=False)

#### Artikel mit extremen Sentimentwerten (BPWS)

In [ ]:
# Bestimme die 3 Monate mit den höchsten Sentimentwerten
top_3_months = bpw_monthly.nlargest(3, 'BPW Score')[['Datum', 'BPW Score']]

# Bestimme die 3 Monate mit den niedrigsten Sentimentwerten
bottom_3_months = bpw_monthly.nsmallest(3, 'BPW Score')[['Datum', 'BPW Score']]

# Ausgabe der Ergebnisse
print("Top 3 Monate mit höchsten Sentimentwerten:")
print(top_3_months)

print("\nTop 3 Monate mit niedrigsten Sentimentwerten:")
print(bottom_3_months)

### Grafik erstellen

Plotten der mit dem BPWS ermittelten Senitmentsentwicklung in den Artikeln (monatlich und quartalsweise)

In [ ]:
# Konvertieren der Quartale und Monate in datetime-Format
bpw_quarterly['Datum'] = pd.PeriodIndex(
    bpw_quarterly['Jahr'].astype(int).astype(str) + 'Q' + bpw_quarterly['Quartal'].astype(int).astype(str), freq='Q' # Convert 'Jahr' and 'Quartal' to integers before converting to strings
).to_timestamp()

bpw_monthly['Datum'] = pd.to_datetime(bpw_monthly['Jahr'].astype(int).astype(str) + '-' + bpw_monthly['Monat'].astype(str).str.zfill(2) + '-01')

# Filtern der bpw-Daten
end_date = pd.to_datetime("2021-09-30")
bpw_quarterly = bpw_quarterly[bpw_quarterly['Datum'] <= end_date]
bpw_monthly = bpw_monthly[bpw_monthly['Datum'] <= end_date]

# Erstellen des Plots
plt.figure(figsize=(12, 6))

# Plot für quartalsweises bpw
plt.plot(bpw_quarterly['Datum'], bpw_quarterly['BPW Score'], label='BPW Sentiment der Artikel (quartalsweise)', color='blue')

# Plot für monatliches Sentiment
plt.plot(bpw_monthly['Datum'], bpw_monthly['BPW Score'], label='BPW Sentiment der Artikel (monatlich)', color='cornflowerblue')

# Horizontale Linie für Neutralität
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=bpw_quarterly['Datum'].min(),
                           end=bpw_quarterly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Achsentitel und Beschriftungen
plt.title('', fontsize=14)
# plt.xlabel('Quartal', fontsize=12)
plt.ylabel('Sentiment-Score', fontsize=12)
plt.grid(alpha=0.3)
# Einfügen der Legende
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
plt.tight_layout()

# Speichern der Artikel
plt.savefig('artikel_bpw_scoring.png', format='png', dpi=600)

plt.show()

### Vergleich mit ifo-Geschäftklimaindex

In [ ]:
# Definieren der Ereignisse und deren Zeiträume in datetime
events = {
    # "Dotcom-Blase": ("2001-02-01", "2003-06-30"),
    "Weltwirtschaftskrise": ("2008-01-01", "2009-04-30"),
    "Coronakrise": ("2019-09-01", "2020-04-30")
}

Monatlich

In [ ]:
bpw_monthly = pd.read_csv('artikel_bpws_relevant_monat.csv')

In [ ]:
# Begrenzen der Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")
start_date = pd.to_datetime("2005-01-01")

# Datei mit ifo- Geschäftsklimaindex laden
file_path = "ifo-Geschäftsklima.xlsx"
ifo_data = pd.read_excel(file_path)

# Konvertieren der Spalte "Monat/Jahr" in datetime-Format
ifo_data['Datum'] = pd.to_datetime(ifo_data['Monat/Jahr'], format=' %m/%Y')

# Sortieren der Daten nach Datum
ifo_data = ifo_data.sort_values(by='Datum')
ifo_data = ifo_data[ifo_data['Datum'] <= end_date]

# Konvertieren der Quartale und Monate in datetime-Format
bpw_monthly['Datum'] = pd.to_datetime(
    bpw_monthly['Jahr'].astype(int).astype(str) + '-' + bpw_monthly['Monat'].astype(str).str.zfill(2) + '-01'
)

# Filtern der bpw-Daten
bpw_monthly = bpw_monthly[(bpw_monthly['Datum'] >= start_date) & (bpw_monthly['Datum'] <= end_date)]

# Erstellen der Grafik
fig, ax1 = plt.subplots(figsize=(12, 6))

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=bpw_monthly['Datum'].min(),
                           end=bpw_monthly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Plot für monatliches Sentiment
line1, = ax1.plot(bpw_monthly['Datum'], bpw_monthly['BPW Score'], label='BPWS Sentiment der Artikel (monatlich)', color='cornflowerblue')
ax1.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax1.set_ylabel('Sentiment-Score', color='cornflowerblue')
ax1.tick_params(axis='y', labelcolor='cornflowerblue')
ax1.grid(alpha=0.3)

# Plot für den ifo-Geschäftsklimaindex
ax2 = ax1.twinx()
line2, = ax2.plot(ifo_data['Datum'], ifo_data['geschäftsklima-veränderung'],
                  label='ifo-Geschäftsklimaindex (monatlich)', color='tomato')
ax2.set_ylabel('ifo-Geschäftsklimaindex', color='tomato')
ax2.tick_params(axis='y', labelcolor='tomato')

for event, (start_date, end_date) in events.items():
    ax1.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3, label=f'{event} (Krise)')

# Kombinieren der Legenden
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

# Titel und Layout
plt.title('', fontsize=14)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_ifo_scoring_m.png', format='png', dpi=600)

# Zeige den Plot
plt.show()

Quartalsweise

In [ ]:
bpw_quarterly = pd.read_csv('artikel_bpws_relevant_quartal.csv')

In [ ]:
# Begrenze die Daten auf den Zeitraum bis Ende Q3 2021
end_date = pd.to_datetime("2021-09-30")
start_date = pd.to_datetime("2005-01-01")

# Laden des ifo-Geschäftsklimaincex und konvertieren von 'Datum' in datetime-Format
file_path = "ifo-Geschäftsklima.xlsx"
ifo_data = pd.read_excel(file_path)
ifo_data['Datum'] = pd.to_datetime(ifo_data['Monat/Jahr'], format=' %m/%Y')

ifo_data = ifo_data[ifo_data['Datum'] <= end_date]

# Konvertieren von 'Datum' in Quartale
ifo_data['Quartal'] = ifo_data['Datum'].dt.to_period('Q')

# Aggregieren des Geschäftsklimaindex quartalsweise (Mittelwert pro Quartal)
ifo_quarterly = ifo_data.groupby('Quartal')['geschäftsklima-veränderung'].mean().reset_index()

# Konvertieren der Quartale in Timestamps für den Plot
ifo_quarterly['Datum'] = ifo_quarterly['Quartal'].dt.to_timestamp()

# Konvertieren der Quartale und Monate in datetime-Format
bpw_quarterly['Datum'] = pd.PeriodIndex(
    bpw_quarterly['Jahr'].astype(int).astype(str) + 'Q' + bpw_quarterly['Quartal'].astype(int).astype(str), freq='Q' # Convert 'Jahr' and 'Quartal' to integers before converting to strings
).to_timestamp()

# Filtern der bpw-Daten
bpw_quarterly = bpw_quarterly[(bpw_quarterly['Datum'] >= start_date) & (bpw_quarterly['Datum'] <= end_date)]

# Erstellen der Grafik
fig, ax1 = plt.subplots(figsize=(12, 6))

# Erstellen der xticks (jedes zweite Quartal)
tick_dates = pd.date_range(start=bpw_quarterly['Datum'].min(),
                           end=bpw_quarterly['Datum'].max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]  # Dynamische Quartalslabels

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Plot für monatliches Sentiment
line1, = ax1.plot(bpw_quarterly['Datum'], bpw_quarterly['BPW Score'], label='BPWS Sentiment der Artikel (quartalsweise)', color='blue')
ax1.axhline(0, color='gray', linestyle='--', linewidth=0.8)
ax1.set_ylabel('Sentiment-Score', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(alpha=0.3)

# Plot für den ifo-Geschäftsklimaindex
ax2 = ax1.twinx()
line2, = ax2.plot(ifo_quarterly['Datum'], ifo_quarterly['geschäftsklima-veränderung'], label='ifo-Geschäftsklimaindex (quartalsweise)', color='maroon')
ax2.set_ylabel('ifo-Geschäftsklimaindex', color='maroon')
ax2.tick_params(axis='y', labelcolor='maroon')

for event, (start_date, end_date) in events.items():
    ax1.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3, label=f'{event} (Krise)')

# Kombinieren der Legenden
lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

# Titel und Layout
plt.title('', fontsize=14)
plt.tight_layout()

# Speichern der Grafik
# plt.savefig('artikel_ifo_scoring_q.png', format='png', dpi=600)

# Zeige den Plot
plt.show()

### Korrelation mit der Entwicklung des ifo-Geschäftsklimaindex

In [ ]:
# Konvertieren der Datumsangaben in datetime-Format
bpw_monthly['Datum'] = pd.to_datetime(bpw_monthly['Datum'])
bpw_quarterly['Datum'] = pd.to_datetime(bpw_quarterly['Datum'])
ifo_data['Datum'] = pd.to_datetime(ifo_data['Monat/Jahr'], format=' %m/%Y')

# Filteren des Zeitraums von Januar 2005 bis September 2021
start_date = pd.to_datetime("2005-01-01")
end_date = pd.to_datetime("2021-09-30")

bpw_monthly = bpw_monthly[(bpw_monthly['Datum'] >= start_date) & (bpw_monthly['Datum'] <= end_date)]
bpw_quarterly = bpw_quarterly[(bpw_quarterly['Datum'] >= start_date) & (bpw_quarterly['Datum'] <= end_date)]
ifo_data = ifo_data[(ifo_data['Datum'] >= start_date) & (ifo_data['Datum'] <= end_date)]

# Berechnend der Korrelation für monatliche Daten
monthly_merged = pd.merge(
    bpw_monthly[['Datum', 'BPW Score']],
    ifo_data[['Datum', 'geschäftsklima-veränderung']],
    on='Datum'
)

# Pearson-Korrelation und p-Wert für monatliche Daten
monthly_corr, monthly_p_value = pearsonr(
    monthly_merged['BPW Score'],
    monthly_merged['geschäftsklima-veränderung']
)

# Berechnen der Korrelation für quartalsweise Daten
bpw_quarterly['Quartal'] = bpw_quarterly['Datum'].dt.to_period('Q')
ifo_data['Quartal'] = ifo_data['Datum'].dt.to_period('Q')

quarterly_merged = pd.merge(
    bpw_quarterly[['Quartal', 'BPW Score']],
    ifo_data.groupby('Quartal')['geschäftsklima-veränderung'].mean().reset_index(),
    on='Quartal'
)

# Pearson-Korrelation und p-Wert für quartalsweise Daten
quarterly_corr, quarterly_p_value = pearsonr(
    quarterly_merged['BPW Score'],
    quarterly_merged['geschäftsklima-veränderung']
)

# Ausgabe der Korrelationswerte und p-Werte
print(f"Korrelation (monatlich): {monthly_corr:.2f}, p-Wert: {monthly_p_value}")
print(f"Korrelation (quartalsweise): {quarterly_corr:.2f}, p-Wert: {quarterly_p_value}")

In [ ]:
# Berechnung der Lag-Korrelationen
def calculate_lag_correlation(data, score_col, index_col, target_col, lags):
    correlations = []
    for lag in lags:
        shifted = data.copy()
        shifted[score_col] = shifted[score_col].shift(lag)
        corr = shifted[score_col].corr(shifted[target_col])
        correlations.append({'Lag': lag, 'Correlation': corr})
    return pd.DataFrame(correlations)

# Lags definieren
lags = range(-4, 5)  # Lags von -4 bis +4

# Berechnung der monatlichen Korrelationen
monthly_correlation_data = calculate_lag_correlation(
    data=monthly_merged,
    score_col='BPW Score',
    index_col='Datum',
    target_col='geschäftsklima-veränderung',
    lags=lags
)

# Berechnung der quartalsweisen Korrelationen
quarterly_correlation_data = calculate_lag_correlation(
    data=quarterly_merged,
    score_col='BPW Score',
    index_col='Quartal',
    target_col='geschäftsklima-veränderung',
    lags=lags
)

# Plot der Lag-Analyse (monatlich)
plt.figure(figsize=(12, 6))
plt.plot(monthly_correlation_data['Lag'], monthly_correlation_data['Correlation'], marker='o', color='cornflowerblue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
# plt.title('Lag-Analyse: Monatliche Sentiment- und ifo-Geschäftsklima-Korrelation', fontsize=14)
plt.xlabel('Lag (Monate)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('lag_artikel_relevant_m.png', dpi=600)
plt.show()

# Plot der Lag-Analyse (quartalsweise)
plt.figure(figsize=(12, 6))
plt.plot(quarterly_correlation_data['Lag'], quarterly_correlation_data['Correlation'], marker='o', color='blue')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
# plt.title('Lag-Analyse: Quartalsweise Sentiment- und ifo-Geschäftsklima-Korrelation', fontsize=14)
plt.xlabel('Lag (Quartale)', fontsize=12)
plt.ylabel('Korrelation', fontsize=12)
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig('lag_artikel_relevant_q.png', dpi=600)
plt.show()

## Häufigkeit der Artikel über die Zeit

In [ ]:
from scipy.interpolate import make_interp_spline

### Häufigkeitsverteilung pro Monat

In [ ]:
# Laden der Daten
df_a = pd.read_csv('bereinigte_artikel_spezifisch.csv')
df_r = pd.read_csv('bereinigte_reden_spezifisch.csv')

In [ ]:
# Konvertiere das Datum in datetime-Format
df_a['Datum'] = pd.to_datetime(df_a['Datum'], format='%Y-%m-%d')
# df_r['date'] = pd.to_datetime(df_r['date'], format='%Y-%m-%d')

# Extrahiere Jahr, Quartal und Monat
df_a['datum_monat'] = df_a['Datum'].dt.month
df_a['jahr_quartal'] = df_a['datum_jahr'].astype(str) + 'Q' + df_a['datum_quartal'].astype(str)

# df_r['datum_monat'] = df_r['date'].dt.month
# df_r['jahr_quartal'] = df_r['date_year'].astype(str) + 'Q' + df_r['date_quarter'].astype(str)

# Artikel pro Monat neu berechnen
artikel_pro_monat = df_a.groupby(['datum_jahr', 'datum_monat']).size()
artikel_pro_monat.index = pd.to_datetime([f"{year}-{month}-01" for year, month in artikel_pro_monat.index])

# reden_pro_monat = df_r.groupby(['date_year', 'datum_monat']).size()
# reden_pro_monat.index = pd.to_datetime([f"{year}-{month}-01" for year, month in reden_pro_monat.index])

# Erstellen der xticks für Jahr-Quartal
tick_dates = pd.date_range(start=artikel_pro_monat.index.min(),
                           end=artikel_pro_monat.index.max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]

# Plot
plt.figure(figsize=(12, 6))

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

plt.bar(artikel_pro_monat.index, artikel_pro_monat.values, width=20, color='cornflowerblue')
# plt.bar(reden_pro_monat.index, reden_pro_monat.values, width=20, color='darkseagreen')

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Beschriftungen und Titel
plt.ylabel('Anzahl der Reden', fontsize=12)
# plt.title('Anzahl der Titelseiten pro Monat', fontsize=14)
# plt.legend(loc='upper right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_reden_monat.png', format='png', dpi=600)

plt.show()

### Häufigkeitsverteilung pro Quartal

In [ ]:
# Konvertieren der 'Datum'-Spalte in ein datetime-Format, falls noch nicht geschehen
df_a['Datum'] = pd.to_datetime(df_a['Datum'], format='%Y-%m-%d')

# Extrahieren von Jahr und Quartal aus dem Datum
df_a['datum_jahr'] = df_a['Datum'].dt.year
df_a['datum_quartal'] = df_a['Datum'].dt.quarter

# Kombinieren von Jahr und Quartal zu einer neuen Spalte 'jahr_quartal' im Format YYYYQX
df_a['jahr_quartal'] = df_a['datum_jahr'].astype(str) + 'Q' + df_a['datum_quartal'].astype(str)

# Zählen der Anzahl der Artikel pro Quartal
artikel_pro_quartal = df_a.groupby('jahr_quartal').size()

# Sortieren nach Quartalen
artikel_pro_quartal = artikel_pro_quartal.sort_index()

artikel_pro_quartal.index = pd.PeriodIndex(artikel_pro_quartal.index, freq='Q').to_timestamp()

# Erstellen der xticks für Jahr-Quartal
tick_dates = pd.date_range(start=artikel_pro_quartal.index.min(),
                           end=artikel_pro_quartal.index.max(),
                           freq='2Q')  # Jedes zweite Quartal
tick_labels = [f"{date.year}Q{(date.month // 3) or 4}" for date in tick_dates]

# Plot
plt.figure(figsize=(12, 6))

# Markieren der Ereignisse mit roten Bereichen
for event, (start_date, end_date) in events.items():
    plt.axvspan(pd.to_datetime(start_date), pd.to_datetime(end_date), color='red', alpha=0.3)

# Plot
# plt.bar(artikel_pro_quartal.index, artikel_pro_quartal.values, width=20, color='cornflowerblue')
plt.bar(artikel_pro_quartal.index, artikel_pro_quartal.values, width=80, color='blue')

# Setzen der xticks und Labels
plt.xticks(ticks=tick_dates, labels=tick_labels, rotation=45)

# Beschriftungen und Titel
plt.ylabel('Anzahl der Artikel', fontsize=12)
# plt.title('Anzahl der Titelseiten pro Monat', fontsize=14)
# plt.legend(loc='upper right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# Speichern der Grafik
plt.savefig('artikel_reden_quartal.png', format='png', dpi=600)

plt.show()